In [1]:
from ultralytics import YOLO

import matplotlib.pyplot as plt
import cv2
import sys
import os
import time
import json
import numpy as np
from shapely.geometry import Polygon
from scipy.optimize import linear_sum_assignment

from sam2.build_sam import build_sam2
from sam2.automatic_mask_generator import SAM2AutomaticMaskGenerator
from sam2.sam2_image_predictor import SAM2ImagePredictor
import torch

In [2]:
# loading yolov11 model
model_yolo = YOLO("/Users/JakubSmigielski/all_models/yolo/yolo11x.pt")

In [3]:
# loading sam2.1 model
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

checkpoint = "./checkpoints/sam2.1_hiera_base_plus.pt"
model_cfg = "configs/sam2.1/sam2.1_hiera_b+.yaml"

sam2_model = build_sam2(model_cfg, checkpoint, device=device, apply_postprocessing=False)
predictor = SAM2ImagePredictor(sam2_model)

In [4]:
def segment_yolo_sam2(model_yolo, predictor, image_path, conf=0.05):
    results = model_yolo.predict(image_path, classes=[2, 3, 5, 7], conf=conf)
    image_bgr = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
    predictor.set_image(image_rgb)

    input_boxes = np.array([bbox.data.cpu().numpy()[0][:4] for bbox in results[0].boxes])
    
    if input_boxes.size == 0:
        return []

    masks, _, _ = predictor.predict(
        point_coords=None,
        point_labels=None,
        box=input_boxes,
        multimask_output=False,
    )

    return masks

In [16]:
# merging all car masks in one png

def merged_mask(input_dir, outputdir, height, width):
    if not os.path.exists(output_dir):
            os.makedirs(output_dir)
    
    for image_file in os.listdir(input_dir):
        
        if not image_file.endswith(('jpg', 'png')):
            continue
        print(f"\nImage: {image_file}")
    
        masks = segment_yolo_sam2(model_yolo, predictor, os.path.join(input_dir, image_file), conf = 0.05)

        combined_mask = np.zeros((height, width), dtype=np.uint8)
        
        for mask in masks:
            mask = mask.squeeze(0).astype(np.uint8)  
            combined_mask = np.maximum(combined_mask, mask)

        mask_color = [0, 255, 0]
        
        combined_mask_rgba = np.zeros((height, width, 4), dtype=np.uint8)
        combined_mask_rgba[:, :, :3] = mask_color
        combined_mask_rgba[:, :, 3] = combined_mask * 255
        
        output_mask_path = os.path.join(output_dir, image_file.replace('.jpg', '_merged_mask.png'))
        cv2.imwrite(output_mask_path, combined_mask_rgba)
        
        print(f"Saved combined mask to: {output_mask_path}")

In [17]:
image_bgr = cv2.imread("/Users/JakubSmigielski/ParkingApp/11_13_test/weti2_2024_11_13_09_17.jpg")
height, width, _ = image_bgr.shape

In [18]:
input_dir = "/Users/JakubSmigielski/ParkingApp/dataset_with_mask"
output_dir = "/Users/JakubSmigielski/ParkingApp/output_masks"

merged_mask(input_dir, output_dir, height, width)


Image: weti2_2024_11_13_10_07_m.jpg

image 1/1 /Users/JakubSmigielski/ParkingApp/dataset_with_mask/weti2_2024_11_13_10_07_m.jpg: 384x640 34 cars, 2 trucks, 342.4ms
Speed: 1.7ms preprocess, 342.4ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)
Saved combined mask to: /Users/JakubSmigielski/ParkingApp/output_masks/weti2_2024_11_13_10_07_m_merged_mask.png

Image: weti2_2024_11_13_11_07_m.jpg

image 1/1 /Users/JakubSmigielski/ParkingApp/dataset_with_mask/weti2_2024_11_13_11_07_m.jpg: 384x640 31 cars, 1 truck, 416.3ms
Speed: 1.6ms preprocess, 416.3ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)
Saved combined mask to: /Users/JakubSmigielski/ParkingApp/output_masks/weti2_2024_11_13_11_07_m_merged_mask.png

Image: weti2_2024_11_13_09_57_m.jpg

image 1/1 /Users/JakubSmigielski/ParkingApp/dataset_with_mask/weti2_2024_11_13_09_57_m.jpg: 384x640 32 cars, 1 truck, 361.5ms
Speed: 1.3ms preprocess, 361.5ms inference, 0.6ms postprocess per image at shape (1, 3,

In [19]:
input_mask = "/Users/JakubSmigielski/ParkingApp/output_masks"
output_images = "/Users/JakubSmigielski/ParkingApp/transparency_masks"

In [ ]:
# transparency mask every image and full mask 

def transparency_mask(input_mask, output_images, height, width):
    if not os.path.exists(output_images):
        os.makedirs(output_images)

    transparent_image = np.zeros((height, width, 4), dtype=np.uint8)
    transparent_image[:, :, 3] = 0 

    mask_files = sorted(os.listdir(input_mask))
    for mask_file in mask_files:
        
        if not mask_file.endswith(('jpg', 'png')):
            continue
        print(f"\nMask: {mask_file}")
        
        mask_path = os.path.join(input_mask, mask_file)
        mask = cv2.imread(mask_path, cv2.IMREAD_UNCHANGED)

        # only non-zero values will be added
        mask_nonzero = mask[:, :, 3] > 0

        transparent_image[mask_nonzero] = cv2.addWeighted(transparent_image[mask_nonzero], 1, mask[mask_nonzero], 0.025, 0)

        output_mask_path = os.path.join(output_images, mask_file.replace('_combined_mask.png', '_transparency_mask.png'))
        cv2.imwrite(output_mask_path, transparent_image)
        print(f"Saved to: {output_mask_path}")

    
    treshold = 0.05
    
    full_mask = np.zeros((height, width, 4), dtype=np.uint8)
    alpha_values = transparent_image[:, :, 3]
    mask_threshold = alpha_values > round(treshold * 255)
    
    full_mask[mask_threshold, 3] = 255

    mask_color = [0, 255, 0]
    full_mask[:, :, :3] = mask_color
    
    output_mask_path = os.path.join(output_images, "zzz_full_mask.png")
    cv2.imwrite(output_mask_path, full_mask)
    print(f"Saved to: {output_mask_path}")

In [21]:
transparency_mask(input_mask, output_images, height, width)


Mask: weti2_2024_11_13_09_17_m_merged_mask.png
Saved to: /Users/JakubSmigielski/ParkingApp/transparency_masks/weti2_2024_11_13_09_17_m_merged_mask.png

Mask: weti2_2024_11_13_09_27_m_merged_mask.png
Saved to: /Users/JakubSmigielski/ParkingApp/transparency_masks/weti2_2024_11_13_09_27_m_merged_mask.png

Mask: weti2_2024_11_13_09_37_m_merged_mask.png
Saved to: /Users/JakubSmigielski/ParkingApp/transparency_masks/weti2_2024_11_13_09_37_m_merged_mask.png

Mask: weti2_2024_11_13_09_47_m_merged_mask.png
Saved to: /Users/JakubSmigielski/ParkingApp/transparency_masks/weti2_2024_11_13_09_47_m_merged_mask.png

Mask: weti2_2024_11_13_09_57_m_merged_mask.png
Saved to: /Users/JakubSmigielski/ParkingApp/transparency_masks/weti2_2024_11_13_09_57_m_merged_mask.png

Mask: weti2_2024_11_13_10_07_m_merged_mask.png
Saved to: /Users/JakubSmigielski/ParkingApp/transparency_masks/weti2_2024_11_13_10_07_m_merged_mask.png

Mask: weti2_2024_11_13_10_17_m_merged_mask.png
Saved to: /Users/JakubSmigielski/Parking